In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import argparse

import pandas as pd
from torch.autograd import Variable
import importlib.util
from torch.utils.data import DataLoader
from datasets import Dataset
module_path = r"D:/timeseries/package/state-spaces-simple/src/models/sequence/ss/standalone/s4.py"

spec = importlib.util.spec_from_file_location("S4", module_path)
S4 = importlib.util.module_from_spec(spec)
spec.loader.exec_module(S4)
from torch.optim import AdamW

from sqlalchemy import create_engine
from sqlalchemy import text

CUDA extension for cauchy multiplication not found. Install by going to extensions/cauchy/ and running `python setup.py install`. This should speed up end-to-end training by 10-50%
Falling back on slow Cauchy kernel. Install at least one of pykeops or the CUDA extension for efficiency.


In [61]:
data=pd.read_csv('D:/timeseries/data/data_6.csv')


In [62]:
data

,SYMBOL,DATE,TIME,PRICE,SIZE
0,AAPL,2014-01-02,4:00:00,561.02,9
1,AAPL,2014-01-02,4:00:00,560.00,5
2,AAPL,2014-01-02,4:00:00,558.19,86
3,AAPL,2014-01-02,4:00:00,558.19,64
4,AAPL,2014-01-02,4:00:00,557.00,5
...,...,...,...,...,...
86035742,MSFT,2014-12-31,19:08:52,46.48,50
86035743,MSFT,2014-12-31,19:12:03,46.48,7
86035744,MSFT,2014-12-31,19:37:51,46.57,100
86035745,MSFT,2014-12-31,19:52:13,46.57,100


In [72]:
names=list(set(data["SYMBOL"]))

In [91]:
def transform_data(sel,names):
  
  datamain=sel.query("SYMBOL=='%s'"%(names[0]))
  datamain=datamain[["DATE","PRICE"]]
  datamain.drop_duplicates(subset=["DATE"], keep='first', inplace=True)
  datamain=datamain.sort_values(by="DATE")
  s=names[0]+"_price"
  s2=names[0]+"_size"
  # datamain["DATE"]=pd.to_datetime(datamain["DATE"])
  datamain=datamain.rename(columns={"PRICE":s})
  # datamain[s] = pd.to_numeric(datamain["PRICE"])
  # datamain[s2] = pd.to_numeric(datamain["SIZE"])


  for name in names:
    if name==names[0]:
      continue
    data=sel.query("SYMBOL=='%s'"%name)

    data=data[["DATE","PRICE"]]
    data.drop_duplicates(subset=["DATE"], keep='first', inplace=True)
    
    # data["PRICE"]=pd.to_datetime(data["PRICE"])
    # data["Close_"] = pd.to_numeric(data["Close_"])
 
    data=data.rename(columns={"PRICE": "%s_price"%name})


    datamain=pd.merge(datamain,data,on="DATE", how="outer")
  return datamain

In [92]:
new_data=transform_data(data,names)

In [94]:
def normalize(data):
    data=data.astype(float)
    mean_list=[]
    std_list=[]
    for i in data.columns:
        try:
          # mean=data[i].mean()
          mean=0
        except:
          print(data[i],i)
          break
        std=1
        std=data[i].std()
        data[i]=(data[i]-mean)/std
        # for j in range(len(data[i])):
          # if data[i][j]!=0:
          #   first=data[i][j]
          #   data[i]=data[i]/first+5
          #   break

        
        mean_list.append(mean)
        std_list.append(std)   
    # return data,first
    return data,mean_list,std_list

def get_mask(data):
    """
    data should in the form of pd.df
    gen a tenor with 0 and 1 to represent missing data
    """
    mask = ~data.isnan().values
 
    mask_tensor = torch.tensor(mask, dtype=torch.float32)
    
    mask_tensor= mask_tensor.transpose(0,1)
    return mask_tensor

def mape(A,F,maskf_sub):
  sum=0
  length=0
  for i in range(len(A)):
   
    if maskf_sub[i]!=0:
      sum+=abs(A[i] - F[i]) / abs(A[i])
      length+=1
  if length>0:
    return 100/length*sum
  
  return 0
def smape(A, F,maskf_sub):
  sum=0
  length=0
  for i in range(len(A)):
    if maskf_sub[i]!=0:
      sum+=2 * abs(F[i] - A[i]) / (abs(A[i]) + abs(F[i]))
      length+=1
  if length>0:

    return 100/length * sum
  return 0

In [95]:

def create_inputs(data, context_length, prediction_length):
    num_days, num_products = data.shape
    num_samples = num_days - context_length - prediction_length + 1
   

    samples = torch.zeros((num_samples, context_length,num_products))

    for i in range(num_samples):
        samples[i,:,:] = data[i:i+context_length]

    return samples,num_samples
def create_targets(data,context_length,prediction_length):
    num_days, num_products = data.shape
    num_samples = num_days - context_length - prediction_length + 1
    

    targets = torch.zeros((num_samples, prediction_length,num_products))

    for i in range(num_samples):
        targets[i, :,:] = data[i+context_length:i+prediction_length+context_length]

    return targets




def split_train_val(data,prediction_period,batchs,context_length,col_len):
 
    whole,m,std=normalize(data.iloc[:,1:])
    # whole,m,std=normalize(data)
    whole=torch.tensor(whole.values)
    # whole=whole.transpose(0,1)
   
    inputs,period=create_inputs(whole,context_length,prediction_period)
    target=create_targets(whole,context_length,prediction_period)
    inputs=inputs.reshape(period,1,col_len*context_length)
    target=target.reshape(period,1,prediction_period*col_len)
    # print(inputs.shape)
    # train_input=inputs[:period-prediction_period].transpose(0,1)
    # test_input=inputs[:period-prediction_period].transpose(0,1)
    # train_target=target[:period-prediction_period].transpose(0,1)
    # test_target=target[period-prediction_period:].transpose(0,1)
    train_input=inputs[:period-prediction_period]
    test_input=inputs[-1:]
    train_target=target[:period-prediction_period]
    test_target=target[-1:]
    print(test_input.shape,test_target.shape)
    
    traindict={'target':train_target,'input':train_input}
    testdict={'target':test_target,'input':test_input}
    train=Dataset.from_dict(traindict)
    train=train.with_format('torch')
    test=Dataset.from_dict(testdict)
    test=test.with_format('torch')
    

    train_loader = DataLoader(train, batch_size=batchs, shuffle=False)
    test_loader = DataLoader(test, batch_size=batchs, shuffle=False)
    return train_loader, test_loader,m,std

class S4Model(nn.Module):

    def __init__(
        self, 
        d_input, 
        d_output=10, 
        d_model=256, 
        n_layers=4, 
        dropout=0.2,
        prenorm=False,
    ):
        super().__init__()

        self.prenorm = prenorm

        # Linear encoder (d_input = 1 for grayscale and 3 for RGB)
        self.encoder = nn.Linear(d_input, d_model)

        # Stack S4 layers as residual blocks
        self.s4_layers = nn.ModuleList()
        self.norms = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        for _ in range(n_layers):
            self.s4_layers.append(
                S4.S4(
                    d_model=d_model, 
                    l_max=1024, 
                    bidirectional=True,
                    postact='glu',
                    dropout=dropout, 
                    transposed=True,
                )
            )
            self.norms.append(nn.LayerNorm(d_model))
            self.dropouts.append(nn.Dropout2d(dropout))

        # Linear decoder
        self.decoder = nn.Linear(d_model, d_output)

    def forward(self, x):
        """
        Input x is shape (B, L, d_input)
        """
        x = self.encoder(x)  # (B, L, d_input) -> (B, L, d_model)
        
        x = x.transpose(-1, -2)  # (B, L, d_model) -> (B, d_model, L)
        for layer, norm, dropout in zip(self.s4_layers, self.norms, self.dropouts):
            # Each iteration of this loop will map (B, d_model, L) -> (B, d_model, L)

            z = x
            if self.prenorm:
                # Prenorm
                z = norm(z.transpose(-1, -2)).transpose(-1, -2)
            
            # Apply S4 block: we ignore the state input and output
            z, _ = layer(z)

            # Dropout on the output of the S4 block
            z = dropout(z)

            # Residual connection
            x = z + x

            if not self.prenorm:
                # Postnorm
                x = norm(x.transpose(-1, -2)).transpose(-1, -2)

        x = x.transpose(-1, -2)

        # Pooling: average pooling over the sequence length
        x = x.mean(dim=1)

        # Decode the outputs
        x = self.decoder(x)  # (B, d_model) -> (B, d_output)

        return x

In [96]:
def train_model(epoch,train,ct,pt,col_len):
    model = S4Model(
    d_input=col_len*ct, 
    d_output=pt*col_len, 
    d_model=64, 
    n_layers=4, 

    dropout=0.1,
    prenorm=False
    )
    device='cpu'
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=0.001,)

    model.train()
    for epoch in range(epoch):
        for ind,batch in enumerate(train):
            optimizer.zero_grad()
            target=batch['target']
            inputs=batch['input']
            # target_mask = ~torch.isnan(target)
            # target_mask = target_mask.view(target.shape[0],target.shape[1],target.shape[2])
            # valid_target = target[target_mask]
            # input_mask=target_mask.reshape()
            # valid_inputs = inputs[target_mask]
        
            
            # inputs=Variable(inputs,requires_grad=True)
            # input_mask = ~torch.isnan(inputs)
            # input_mask=input_mask.reshape(inputs.shape[0],inputs.shape[1],inputs.shape[2])
            # print(input_mask.shape)
            # valid_inputs = inputs[input_mask]
            # print(valid_inputs.shape)
            
            # inputs=inputs.nan_to_num()
            inputs=Variable(inputs,requires_grad=True)
            
            outputs = model(inputs)
            # print(outputs)
            # output_mask=target_mask.reshape(outputs.shape[0],outputs.shape[1])
            criterion = nn.MSELoss()
            # valid_outputs=outputs[output_mask]
            # print(valid_outputs)
            # break
            loss =criterion(outputs,target)
            # loss =criterion(valid_outputs,valid_target)

            loss.backward()
            optimizer.step()
        
        if epoch % 2 == 0:
            print(loss.item())

        model.eval()
    return model

In [97]:
def predict(model,test,pt,col_len):
    pred=[]
    for ind,batch in enumerate(test):
        inputs=batch['input']
        inputs=inputs.nan_to_num()
        print(inputs,inputs.shape)
        out=model(inputs)
        out=out.reshape(pt,col_len)
        out=out.T
        pred=pred+list(out)
        
    return pred
            
def denormalize(data,mean,std):
    for i in range(len(data)):
        print(data[i],std[i],mean[i])
        data[i]=data[i]*std[i]+mean[i]
    return data       


In [98]:
def S4_run(data,pred_length,batch_size,context_l,epoch,col_len):
    train,test,m,std=split_train_val(data,pred_length,batch_size,context_l,col_len)
    model=train_model(epoch,train,context_l,pred_length,col_len)
    pred=predict(model,test,pred_length,col_len)
    denormalize_pred=denormalize(pred,m,std)
    return model,pred,denormalize_pred,test,m,std
    

    

In [126]:

get=S4_run(new_data,1,50,2,50,3)


torch.Size([1, 1, 6]) torch.Size([1, 1, 3])


d:\Users\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([50, 1, 3])) that is different to the input size (torch.Size([50, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
d:\Users\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([49, 1, 3])) that is different to the input size (torch.Size([49, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


38.81801986694336
24.98859214782715
18.627809524536133
14.692171096801758
11.717824935913086
9.26452922821045
7.213115215301514
5.540320873260498
4.213038444519043
3.1828267574310303
2.4053151607513428
1.8364379405975342
1.4290872812271118
1.1391205787658691
0.9306294918060303
0.7767571210861206
0.659277617931366
0.5658623576164246
0.48564326763153076
0.41251298785209656
0.34935280680656433
0.3005467355251312
0.265882670879364
0.24185191094875336
0.22467705607414246
tensor([[[ 0.5069,  1.9837, 12.2360,  0.5060,  1.9692, 12.1183]]]) torch.Size([1, 1, 6])
tensor([0.5391], grad_fn=<UnbindBackward0>) 225.30781514157803 0
tensor([1.9446], grad_fn=<UnbindBackward0>) 268.3608860827003 0
tensor([11.4354], grad_fn=<UnbindBackward0>) 3.9097805695284733 0


In [101]:

def mape(A,F,maskf_sub):
  sum=0
  length=0
  for i in range(len(A)):
    # print(A[i],"A",maskf_sub[i],"ma")
    if not torch.isnan(A[i]):
    # if maskf_sub[i]!=0:
      sum+=abs(A[i] - F[i]) / abs(A[i])
      length+=1
  if length>0:
    return 100/length*sum
  
  return 0
def smape(A, F,maskf_sub):
  sum=0
  length=0
  for i in range(len(A)):
    if not torch.isnan(A[i]):
    # if maskf_sub[i]!=0:
      sum+=2 * abs(F[i] - A[i]) / (abs(A[i]) + abs(F[i]))
      length+=1
  if length>0:

    return 100/length * sum
  return 0

In [18]:
act_mask

tensor([ True,  True,  True,  ..., False,  True,  True])

In [133]:
smpl=[]
mpl=[]
m=get[4]
std=get[5]
act=next(iter(get[3]))
act=(act['target']).reshape(1,3)
act=act.T
deno_pred=get[2]
act=denormalize(act,m,std)
act_mask=(~torch.isnan(act))
act_mask=act_mask.reshape(3*1)



for i in range(3):

    mp=mape(act[i],deno_pred[i],act_mask)
    smp=smape(act[i],deno_pred[i],act_mask)
    
    smpl.append(float(smp))
    mpl.append(float(mp))

tensor([0.5010]) 225.30781514157803 0
tensor([1.9820]) 268.3608860827003 0
tensor([12.0339]) 3.9097805695284733 0


In [134]:
for i in range(len(deno_pred)):
    for j in range(len(deno_pred[i])):
        deno_pred[i][j]=float(deno_pred[i][j])
deno_pred


[tensor([121.4694], grad_fn=<CopySlices>),
 tensor([521.8474], grad_fn=<CopySlices>),
 tensor([44.7098], grad_fn=<CopySlices>)]

In [136]:
new_pred=[]
for i in range(1):
    new_pred.append([])
    for j in range(3):
        new_pred[i].append(float(deno_pred[j][i]))

In [137]:
act=act.transpose(0,1)
act.shape

torch.Size([1, 3])

In [138]:
# act=act.T
result= pd.DataFrame({'S4_mape':mpl,'S4_smape':smpl})
result.to_csv('D:/timeseries/result/temp_2.csv')
res2= pd.DataFrame({'name':names,'day1/1_pred':act[0], 'day1/1_actu':new_pred[0]
                       })
res2.to_csv('D:/timeseries/result/temp.csv')

PermissionError: [Errno 13] Permission denied: 'D:/timeseries/result/temp.csv'

In [119]:
# act=act.T
result= pd.DataFrame({'S4_mape':mpl,'S4_smape':smpl})
result.to_csv('D:/timeseries/result/temp_2.csv')
res2= pd.DataFrame({'name':names,'day1/7_pred':act[0], 'day1/7_actu':new_pred[0],'day2/7_pred':act[1], 
                    'day2/7_actu':new_pred[1],'day3/7_pred':act[2], 'day3/7_actu':new_pred[2],'day4/7_pred':act[3], 'day4/7_actu':new_pred[3]
                    ,'day5/7_pred':act[4], 'day5/7_actu':new_pred[4],'day6/7_pred':act[5], 'day6/7_actu':new_pred[5],
                    'day7/7_pred':act[6], 'day7/7_actu':new_pred[6]
                       })
res2.to_csv('D:/timeseries/result/temp.csv')